In [ ]:
# pip install huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 kB 380.4 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
from huggingface_hub import hf_hub_download
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import json
import torch

print("torch version:", torch.__version__)
print("torch cuda version:", torch.version.cuda)
print("torch cuda available:", torch.cuda.is_available())

torch version: 2.1.2
torch cuda version: None
torch cuda available: False


In [ ]:
import pandas as pd
# the call takes 16min for this 3GB training + 500mb dev + 500mb test dataset, so we are using a local copy
# https://huggingface.co/datasets/kensho/DocFinQA
splits = {'train': 'train.json', 'validation': 'dev.json', 'test': 'test.json'}
df = pd.read_json("hf://datasets/kensho/DocFinQA/" + splits["train"])
df

,Context,Question,Program,Answer
0,# UNITED\tSTATES\n\n# SECURITIES\tAND\tEXCHANG...,what is the the interest expense in 2009?,interest_expense = 100 / 100\nannual_interest_...,380
1,# UNITED\tSTATES\tSECURITIES\tAND\tEXCHANGE\tC...,what was the total operating expenses in 2018 ...,,41932
2,# UNITED\tSTATES\tSECURITIES\tAND\tEXCHANGE\tC...,what percentage of total cash and investments ...,available_for_sale_investments = 14001\ntotal_...,53%
3,# UNITED\tSTATES\n\n# SECURITIES\tAND\tEXCHANG...,what is the growth rate in net revenue in 2008?,net_revenue_2008 = 959.2\nnet_revenue_2007 = 9...,-3.2%
4,10-K\t1\tcitigroup_10k.htm\tANNUAL\tREPORT\n\n...,what was the growth rate of the loans held-for...,,56.25%
...,...,...,...,...
5730,Table\tof\tContents\n\nUNITED\tSTATES\n\n# SEC...,what was the operating margin from printing pa...,margin_2012 = 599\nmargin_total = 6230\nmargin...,9.6%\\n
5731,UNITED\tSTATES\n\n# SECURITIES\tAND\tEXCHANGE\...,what was the ratio of the debt reduction to th...,debt_reduction = 6.2\n stock_repurchase = 1....,4.43
5732,Table\tof\tContents\n\n# UNITED\tSTATES\tSECUR...,what is the mathematical range of dilutive sha...,dilutive_shares = 4972\n non_dilutive_shares...,562
5733,Table\tof\tContents\n\n# UNITED\tSTATES\n\n# S...,what is the percentage of powertrain systems s...,,14.28%
